In [2]:
from modules.submodules import executing_requete_sparql
from rdflib import Graph, URIRef, Namespace
import random

In [3]:
kesaio = Namespace("http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/")
common_core_ns = Namespace("http://www.ontologyrepository.com/CommonCoreOntologies/")
hasRole = URIRef("http://purl.obolibrary.org/obo/RO_0000087")
participatesIn = URIRef("http://purl.obolibrary.org/obo/RO_0000056")

In [4]:
g = Graph()
g.parse("simulationPlanOrsec/Ontologie/ontologie_enrichie.rdf")

<Graph identifier=N6703df8b20ee4efca830d0f3201469cc (<class 'rdflib.graph.Graph'>)>

# Questions de Compétences

##### What is the nature of the disaster <X>?

In [5]:
requete_sparql = """
SELECT ?type ?description ?seisme
WHERE {
    ?seisme rdf:type ?type.
    ?seisme kesaio:description ?description .
}
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio)
for resultat in resultats:
    print(resultat.seisme.split("/")[-1])
    classe_nom = resultat.type.split("/")[-1]
    print("Nature:", classe_nom)
    print("Description:", resultat.description)

Séisme_de_2018_à_Hokkaidō
Nature: Séisme
Description: séisme_au_Japon_en_2018


##### When did the disaster <X> take place?

In [6]:
requete_sparql = """
SELECT ?jour ?mois ?annee
WHERE {
    ?seisme cco:occurs_on ?date.
    ?date kesaio:has_day ?jour .
    ?date kesaio:has_month ?mois .
    ?date kesaio:has_year ?annee .
}
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
for resultat in resultats:
    jour = str(resultat.jour.split('/')[-1])
    mois = str(resultat.mois.split('/')[-1])
    annee = str(resultat.annee.split('/')[-1])

    print("Jour:", jour)
    print("Mois:", mois)
    print("Année:", annee)

Jour: jour_6
Mois: mois_9
Année: 2018


#### Where did the disaster <X> take place?

In [7]:
requete_sparql = """
SELECT ?city ?pays
WHERE {
    ?seisme cco:occurs_at ?city.
    ?city cco:spatial_part_of ?pays
}
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
for resultat in resultats:
    city = str(resultat.city.split('/')[-1])
    pays = str(resultat.pays.split('/')[-1])

    print("Ville:", city)
    print("Pays:", pays)

Ville: sous-préfecture_Iburi
Pays: Japon


#### What is the criticality level of the disaster <X>?

In [8]:
requete_sparql = """
SELECT ?sévérité ?blessés ?morts
WHERE {
    ?seisme kesaio:nombre_de_morts ?morts .
    ?seisme kesaio:nombre_de_blessés ?blessés .
    ?seisme kesaio:sévérite_de_la_catastrophe ?sévérité .
}
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
for resultat in resultats:
    sévérité = str(resultat.sévérité.split('/')[-1])
    blessés = str(resultat.blessés.split('/')[-1])
    morts = str(resultat.morts.split('/')[-1])

    print("Sévérité de la catastrophe:", sévérité)
    print("Nombre de blessés:", blessés)
    print("Nombre de morts:", morts)

Sévérité de la catastrophe: Niveau 1
Nombre de blessés: 7
Nombre de morts: 2


#### Which ERs are involved in the operation <X>?

In [9]:
requete_sparql = """
    SELECT DISTINCT ?secouriste ?organisation ?role ?identifiant ?funcType 
    WHERE {
        ?x rdf:type/rdfs:subClassOf* kesaio:Catastrophe.
        ?x kesaio:trigger_to ?y.
        ?y cco:has_process_part ?z.
        ?z cco:has_process_part ?w.
        ?secouriste <http://purl.obolibrary.org/obo/RO_0000056> ?w.
        ?secouriste kesaio:identifiant ?identifiant.
        ?secouriste cco:is_affiliated_with ?organisation.
        ?secouriste <http://purl.obolibrary.org/obo/RO_0000087> ?role.
        ?secouriste kesaio:aPourFonction ?func.
        ?func rdf:type ?funcType. 
        # ?funcType rdfs:comment ?funcTypeComment.
        FILTER (?funcType != owl:NamedIndividual)
        FILTER (?funcType != owl:Class)
    }
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
resultats_par_organisation = {}
for row in resultats:
    secouriste = row["secouriste"].split('/')[-1]
    organisation = row["organisation"].split('/')[-1]
    role = row["role"].split('/')[-1]
    identifiant = row["identifiant"].split('/')[-1]
    secouriste_nom = secouriste.split("/")[-1]  
    organisation_nom = organisation.split("/")[-1]  
    specialite = row["funcType"].split('/')[-1]
    role_nom = role.split("/")[-1]  
    if organisation_nom not in resultats_par_organisation:
        resultats_par_organisation[organisation_nom] = {}
    if secouriste_nom not in resultats_par_organisation[organisation_nom]:
        resultats_par_organisation[organisation_nom][secouriste_nom] = {}
    if role_nom not in resultats_par_organisation[organisation_nom][secouriste_nom]:
        resultats_par_organisation[organisation_nom][secouriste_nom][role_nom] = {"identifiant": identifiant, "specialites": []}
    resultats_par_organisation[organisation_nom][secouriste_nom][role_nom]["specialites"].append(specialite)

for organisation, secouristes in resultats_par_organisation.items():
    print("Organisation:", organisation)
    for secouriste, roles in secouristes.items():
        print("  Secouriste:", secouriste)
        for role, info in roles.items():
            print("    Rôle:", role)
            print("    Identifiant:", info["identifiant"])
            print("    Spécialité:", ', '.join(info["specialites"]))
            print("")

Organisation: sapeurPompier
  Secouriste: Pompier6
    Rôle: equipierInter
    Identifiant: d5eae3e1-3768-4094-9aab-d26ece159788
    Spécialité: Rechercher_des_victimes_en_cas_de_catastrophe

  Secouriste: Pompier5
    Rôle: equipierInter
    Identifiant: 7fa3e8b6-76f8-4ae6-94e9-97266de306c5
    Spécialité: Rechercher_des_victimes_en_cas_de_catastrophe

Organisation: gendarmerie
  Secouriste: OfficierRamassage0
    Rôle: uniteDeRecherche
    Identifiant: 1aa63ee3-ef15-4c3d-8e1b-76839b98ec94
    Spécialité: Recherche_et_sauvetage

  Secouriste: OfficierRamassage1
    Rôle: uniteDeRecherche
    Identifiant: e511d21e-67f7-4566-b492-fed68f9f308e
    Spécialité: Recherche_et_sauvetage

Organisation: policeNationale
  Secouriste: Policier0
    Rôle: encadrementEtapplication
    Identifiant: cc7dd5fd-f309-4d38-acac-f6484bb6bc85
    Spécialité: Assistance_administrative

Organisation: samu
  Secouriste: Médecin_urgentiste_0
    Rôle: officierRassemblement
    Identifiant: efc2bac1-b00f-4810-be

#### Where was the advanced medical post of <X>’s healthcare units located?

In [10]:
requete_sparql = """
SELECT ?posteMedical ?latitude ?longitude
WHERE {
    ?posteMedical a kesaio:Poste_médical_avancé.
    ?posteMedical kesaio:latitude ?latitude.
    ?posteMedical kesaio:longitude ?longitude.
}
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
for row in resultats:
    poste_medical = row["posteMedical"].split("/")[-1]  
    latitude = row["latitude"]
    longitude = row["longitude"]
    print("Poste médical avancé :", poste_medical)
    print("Latitude :", latitude)
    print("Longitude :", longitude)
    print("\n")

Poste médical avancé : Poste_médical_avancé_1
Latitude : 42.66203292626276
Longitude : 141.93206812466875




#### Who was the operational commander of the <X> operation?

In [11]:
requete_sparql = """
SELECT ?roleCommandant ?commandant ?typeCommandant
WHERE {
    kesaio:Opération_ kesaio:estSuperviséPAr ?roleCommandant.
    ?commandant <http://purl.obolibrary.org/obo/RO_0000087> ?roleCommandant .
    ?commandant rdf:type ?typeCommandant.
    FILTER (?typeCommandant != owl:NamedIndividual)
    FILTER (?typeCommandant != owl:Class)
}
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
for row in resultats:
    role_commandant = row["roleCommandant"].split("/")[-1]
    commandant = row["commandant"].split("/")[-1] 
    type_commandant = row["typeCommandant"].split("/")[-1] 

    print("Commandant :", commandant)
    # print("Rôle :", role_commandant)
    print("Type :", type_commandant)

Commandant : Commandant_des_operations_de_secours_
Type : Pompier
Commandant : s2
Type : Membre_SAMU
Commandant : Commandant_de_la_Gendarmerie
Type : Gendarme
Commandant : Commandant_de_la_police
Type : Policier


#### Who was the public authority that commanded the <X> operation?

In [12]:
requete_sparql = """
SELECT ?x ?xType
WHERE {
    kesaio:Opération_ kesaio:estSuperviséPAr ?cos.
    ?cos kesaio:estSuperviséPAr ?dsi.
    ?dsi kesaio:estSuperviséPAr ?dsis.
    ?x kesaio:supervise ?dsis.
    ?x rdf:type ?xType.
    FILTER (?xType != owl:NamedIndividual)
}
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio)
for row in resultats:
    autorite = row["x"].split("/")[-1]
    autoriteType = row["xType"].split("/")[-1]
    print("Instance : ",autorite)
    print("Role : ", autoriteType)

Instance :  p
Role :  Préfét


#### What were the actions of the ER [Y] in the operation <X>?

In [13]:
requete_sparql = """
SELECT DISTINCT ?action
WHERE {
    ?x rdf:type/rdfs:subClassOf* cco:OrganizationMember.
    ?x <http://purl.obolibrary.org/obo/RO_0000056> ?action.
}
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
for row in resultats:
    actions = row["action"].split("/")[-1]

    print("Actions des ERs :", actions)

Actions des ERs : Acte_de_ramassage_des_victimes_1
Actions des ERs : Acte_de_secours_des_victimes_1
Actions des ERs : Acte_devacuation_des_victimes_1


#### What is the definition of the term absolute emergency?

In [14]:
requete_sparql = """
SELECT ?comment
WHERE {
    # {kesaio:UA rdfs:comment ?comment.}
    # UNION
    {kesaio:P0 rdfs:comment ?comment.}
}
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
for row in resultats:
    comment = row["comment"]
    print("Commentaire :", comment)


Commentaire : Déclenchement SMUR et ensuite régulation priorité
SMUR = structure mobile d’urgence et de réanimation


#### Who is competent to search and rescue persons drowning as a result of <X>?

In [15]:
requete_sparql = """
SELECT ?equipePlongee
WHERE {
    ?equipePlongee rdf:type/rdfs:subClassOf* kesaio:Equipe_de_plongée.
}
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
for row in resultats:
    equipePlongee = row["equipePlongee"].split("/")[-1]
    print("Equipe :", equipePlongee)


Equipe : equipePlongée


#### How many people were affected by <X>?

In [16]:
requete_sparql = """
SELECT (COUNT(DISTINCT ?victime) AS ?nombreVictimes) (COUNT(DISTINCT ?secouriste) AS ?nombreSecouristes)
WHERE {
    ?victime rdf:type/rdfs:subClassOf* kesaio:Victime.
    ?secouriste rdf:type/rdfs:subClassOf* cco:OrganizationMember.
    ?secouriste <http://purl.obolibrary.org/obo/RO_0000056> ?action.
}
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
for row in resultats:
    nombre_victimes = row["nombreVictimes"]
    nombre_secouristes = row["nombreSecouristes"]
    
    print("Nombre de victimes :", nombre_victimes)
    print("Nombre de secouristes :", nombre_secouristes)
    print("Nombre de personnes affectées :", nombre_victimes + nombre_secouristes)

Nombre de victimes : 9
Nombre de secouristes : 9
Nombre de personnes affectées : 18


#### What was the state of the victim [W] of <X>?

In [17]:
numero_victime = random.randint(0, int(nombre_victimes)) 
requete_sparql = f"""
SELECT ?etatVictime
WHERE {{
    kesaio:victime_{numero_victime} kesaio:has_status ?etatVictime.
}}
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
for row in resultats:
    etat_victime = row["etatVictime"].split('/')[-1]
    print(f"État de la victime_{numero_victime} : {etat_victime}")

État de la victime_3 : ua


#### What means of transport were used in the operation <X>?

In [18]:
requete_sparql = """
SELECT DISTINCT ?type 
WHERE {
    ?x rdf:type/rdfs:subClassOf* cco:OrganizationMember.
    ?x <http://purl.obolibrary.org/obo/RO_0000056> ?action.
    ?action kesaio:needs ?means.
    ?means rdf:type ?type .
    FILTER (?type != owl:NamedIndividual)
    FILTER (?type != kesaio:Equipe_médicale) 
}
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
print("Moyens des ERs :")
for row in resultats:
    actions = row["type"].split("/")[-1]
    print("     ", actions)


Moyens des ERs :
      Capteurs_de_mouvement
      Détecteurs_de_battements_de_cœur
      Détecteurs_de_chaleur
      Grue
      Marteaux-piqueurs
      Vérin_hydrauliques
      Scie
      Véhicule_de_secours_et_d'assistance_aux_victimes
      Véhicule_de_soutien_sanitaire
      Ambulance


#### What types of means are needed to respond to a forest fire?

In [19]:
requete_sparql = """
SELECT DISTINCT ?objetDeBesoin
WHERE {
    kesaio:Acte_dextinction_dincendie rdfs:subClassOf* ?restriction.
    ?restriction owl:onProperty kesaio:needs.
    ?restriction owl:someValuesFrom ?objetDeBesoin.
}
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
print("Types de moyens nécessaires pour répondre à un incendie de forêt :")
for row in resultats:
    means = row["objetDeBesoin"].split("/")[-1]
    print("     ", means)


Types de moyens nécessaires pour répondre à un incendie de forêt :
      Camion_de_citerne_feux_de_foret
      Fourgon_pompe_tonne


#### What are the required specialties for firefighters in victim retrieval? Toutes les spécialités possible

In [20]:
requete_sparql = """
    SELECT DISTINCT ?func
    WHERE {
        ?func rdfs:subClassOf* kesaio:Organization_Member_Function.
    }
"""
resultats = executing_requete_sparql.executer_requete_sparql(g, requete_sparql, kesaio, common_core_ns)
print("Spécialités possibles:")
for row in resultats:
    actions = row["func"].split("/")[-1]
    print("     ", actions)

Spécialités possibles:
      Organization_Member_Function
      Fonctions_de_Gendarme
      Assistance_aux_victimes
      Contrôle_des_frontières_et_des_migrations
      Recherche_et_sauvetage
      Travail_d'équipe_avec_d'autres_services
      Maintien_de_l'ordre_public
      Interventions_en_cas_d'infractions
      Sécurité_publique_dans_les_zones_rurales
      Surveillance_et_patrouille
      Contrôles_routiers
      Enquêtes_criminelles
      Fonctions_de_Membre_SAMU
      Conseil_et_orientation
      Intervenir_dans_des_milieux_difficiles
      Intervention_en_cas_de_crise
      Diagnostic_des_troubles_mentaux
      Gérer_les_appels_médicaux_critiques
      Transport_médicalisé_des_patients
      Gérer_les_situations_médicales_délicates
      Intervention_en_santé_mentale
      Coordonner_les_transferts_inter-hospitaliers
      Pratiquer_des_gestes_médicaux_avancés
      Utiliser_du_matériel_spécialisé
      Apporter_un_soutien_médical_à_distance
      Participer_à_des_opérations_

In [21]:
# Enregistrez l'ontologie enrichie dans un fichier RDF
g.serialize("/home/mandiaye/LORIA/V2/monOnto/ontologie_enrichie.rdf", format="xml")

<Graph identifier=N6703df8b20ee4efca830d0f3201469cc (<class 'rdflib.graph.Graph'>)>